## couples each vbo to a grid



In [1]:
from shapely.geometry import asPoint
import geopandas as gpd
import pandas as pd
import time
import shapely
import matplotlib.pyplot as plt


In [2]:
def VBOCleaner(data):
    print(data.shape)
    
    # only keep objects with id
    data = data[data["identificatie"].notna()]
    print(data.shape)
    
    # only keep objects which are currently in use # CHECK of dit de juiste zijn
    data = data[data.verblijfsobjectStatus.isin(["Verblijfsobject gevormd", 
                                                 "Verblijfsobject in gebruik",
                                                 "Verblijfsobject in gebruik (niet ingemeten)"])]
    data = data[data.pos.notna()]
    print(data.shape)
    
    data["begindatumTijdvakGeldigheid"] =  pd.to_datetime(data['begindatumTijdvakGeldigheid'], format='%Y%m%d%H%M%S%f', errors='coerce')
    data["einddatumTijdvakGeldigheid"] =  pd.to_datetime(data['einddatumTijdvakGeldigheid'], format='%Y%m%d%H%M%S%f', errors='coerce')
    
    def toPoint(pos):
        return asPoint([float(i) for i in pos.split()[:2]])
    data["pos"] = data.pos.apply(toPoint)
    
    return data

In [3]:
vbo = pd.read_csv("VBO.csv", sep = ";", encoding = "utf-8", na_values = "NaN", on_bad_lines = "warn")
vbo = VBOCleaner(vbo)


(19501545, 7)
(18918593, 7)
(18386142, 7)


In [4]:
# # Grid is full eindhoven grid
# grid = pd.read_csv("Nijmegen_grid.csv")
# grid['geometry'] = grid.geometry.apply(shapely.wkt.loads)
# grid = gpd.GeoDataFrame(grid)

In [5]:
def couple_grid_vbo(name):
    t0 = time.time()
    grid = pd.read_csv(name + "_grid.csv")
    grid['geometry'] = grid.geometry.apply(shapely.wkt.loads)
    grid = gpd.GeoDataFrame(grid)
    
    vbo = pd.read_csv("VBO.csv", sep = ";", encoding = "utf-8", na_values = "NaN", on_bad_lines = "warn")
    vbo = VBOCleaner(vbo)
    points = gpd.GeoDataFrame(vbo, geometry = "pos")
    
    
    # join each cbs row with a vbo that contains a vbo point
    pointInPolys = gpd.tools.sjoin(points, grid, op = "within")
    print(time.time() - t0)
    pointInPolys.to_csv(name + "_vbo.csv")

In [6]:
coupled = couple_grid_vbo("Groningen")

(19501545, 7)
(18918593, 7)
(18386142, 7)
1997.0505499839783


In [14]:
coupled.columns

Index(['gebruiksdoelVerblijfsobject', 'oppervlakteVerblijfsobject',
       'verblijfsobjectStatus', 'begindatumTijdvakGeldigheid',
       'einddatumTijdvakGeldigheid', 'identificatie', 'pos', 'index_right',
       'Unnamed: 0', 'C28992R100'],
      dtype='object')

In [17]:
coupled.shape

(197572, 10)

In [16]:
grid.shape

(12566, 3)

In [ ]:
# plot example

plt.plot(*cbs[cbs["C28992R100"] == pointInPolys.iloc[0].C28992R100].geometry.iloc[0].exterior.xy)
plt.scatter(point.x, point.y)